<a href="https://colab.research.google.com/github/nithinv-27/COMSYS-Hackathon-2023---Text-Classification/blob/main/COMSYS_Hackathon_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv("/train.csv")
train.head()

,Unnamed: 0,Text,label
0,0,I can't shake off this constant sense of hopel...,Depression
1,1,I'm constantly second-guessing myself and my d...,Anxiety Disorder
2,2,"I'm feeling physically unwell, but I know it's...",Depression
3,3,I'm desperate to escape the overwhelming fear.,Panic Disorder
4,4,It's hard to describe the sensation of being t...,Panic Disorder


In [ ]:
test=pd.read_csv("/test.csv")

In [ ]:
train_without_id=train.drop('Unnamed: 0', axis=1).copy()

In [ ]:
train_without_id.head()

,Text,label
0,I can't shake off this constant sense of hopel...,Depression
1,I'm constantly second-guessing myself and my d...,Anxiety Disorder
2,"I'm feeling physically unwell, but I know it's...",Depression
3,I'm desperate to escape the overwhelming fear.,Panic Disorder
4,It's hard to describe the sensation of being t...,Panic Disorder


In [ ]:
train_without_id.shape

(785, 2)

###Multi Naive Bayes

In [ ]:
train_without_id_x=train_without_id["Text"]
train_without_id_y=train_without_id["label"]

In [ ]:
test_x=test["Text"]

In [ ]:
i=1
a=dict()
words=train_without_id_y.unique()
for w in words:
  a[w]=i
  i+=1

In [ ]:
a

{'Depression': 1,
 'Anxiety Disorder': 2,
 'Panic Disorder': 3,
 'Anger/ Intermittent Explosive Disorder': 4,
 'Narcissistic Disorder': 5}

In [ ]:
train_without_id_y=train_without_id_y.map(a)

In [ ]:
train_without_id_y

0      1
1      2
2      1
3      3
4      3
      ..
780    1
781    5
782    5
783    5
784    3
Name: label, Length: 785, dtype: int64

In [ ]:
test_x

0      I'm worried about the impact of panic on my re...
1      Trying to maintain my equilibrium when emotion...
2      Feeling like I'm in a constant struggle to fin...
3      I'm quick to dismiss others' opinions as less ...
4      I hate the person I become when I'm consumed b...
                             ...                        
238    The pressure to avoid mistakes can keep me fro...
239    Even when challenges arise, I remain unwaverin...
240    It's like my mind is a radio tuned to the freq...
241    It's hard to explain the surge of panic that c...
242    It's hard to explain the inner conflict of man...
Name: Text, Length: 243, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split
train_x,valid_x,train_y,valid_y=train_test_split(train_without_id_x,train_without_id_y, test_size=0.2, random_state=42)

In [ ]:
train_x

443    Feeling like I'm stuck in a loop of 'what-ifs'...
210    I'm quick to put others down to make myself fe...
133    I'm consumed by worry about things that are be...
741    Some days feel like I'm carrying a heavy weigh...
519    I'm overthinking every little detail and drivi...
                             ...                        
71          I'm struggling to find any sense of purpose.
106    I'm constantly anticipating the next panic att...
270    Feeling like my emotions are on a roller coast...
435    It's hard to describe the inner struggle of ma...
102    It's hard to explain the battle happening with...
Name: Text, Length: 628, dtype: object

In [ ]:
train_y

443    2
210    5
133    2
741    1
519    2
      ..
71     1
106    3
270    4
435    4
102    1
Name: label, Length: 628, dtype: int64

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
stops=stopwords.words('english')

In [ ]:
stops_ls=[]
for words in stops:
  stops_ls.append(words)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(max_features=1500,stop_words=stops_ls)
train_x_vectors=vectorizer.fit_transform(train_x)
valid_x_vectors=vectorizer.transform(valid_x)
test_x_vectors=vectorizer.transform(test_x)

In [ ]:
sorted(vectorizer.vocabulary_,key=lambda x:vectorizer.vocabulary_[x],reverse=True )

In [ ]:
train_x_vectors=train_x_vectors.toarray()

In [ ]:
test_x_vectors=test_x_vectors.toarray()

In [ ]:
test_x_vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
np.shape(test_x_vectors)

(243, 932)

In [ ]:
np.shape(train_x_vectors)

(628, 932)

In [ ]:
len(train_y)

628

In [ ]:
valid_x_vectors=valid_x_vectors.toarray()

In [ ]:
np.shape(valid_x_vectors)

(157, 932)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
clf_mlt = MultinomialNB(alpha=3)
clf_mlt.fit(train_x_vectors,train_y )

MultinomialNB(alpha=3)

In [ ]:
mlt_pred=clf_mlt.predict(valid_x_vectors)

In [ ]:
valid_y

752    3
39     1
211    4
199    5
235    2
      ..
593    1
79     5
148    5
336    3
493    2
Name: label, Length: 157, dtype: int64

In [ ]:
train_pred=clf_mlt.predict(train_x_vectors)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(valid_y, mlt_pred))

              precision    recall  f1-score   support

           1       0.64      0.58      0.61        50
           2       0.59      0.76      0.67        34
           3       1.00      0.43      0.60        21
           4       0.55      0.78      0.65        27
           5       0.90      0.76      0.83        25

    accuracy                           0.66       157
   macro avg       0.74      0.66      0.67       157
weighted avg       0.71      0.66      0.66       157



In [ ]:
print(classification_report(train_y, train_pred))

              precision    recall  f1-score   support

           1       0.92      0.80      0.85       158
           2       0.88      0.93      0.91       119
           3       0.95      0.86      0.90        91
           4       0.82      0.96      0.89       127
           5       0.99      1.00      0.99       133

    accuracy                           0.91       628
   macro avg       0.91      0.91      0.91       628
weighted avg       0.91      0.91      0.91       628



In [ ]:
from sklearn import svm

In [ ]:
clf_svm = svm.SVC(decision_function_shape='ovo')

In [ ]:
clf_svm.fit(train_x_vectors, train_y)

SVC(decision_function_shape='ovo')

In [ ]:
svm_valid_pred=clf_svm.predict(valid_x_vectors)

In [ ]:
svm_train_pred=clf_svm.predict(train_x_vectors)

In [ ]:
print(classification_report(valid_y, svm_valid_pred))

              precision    recall  f1-score   support

           1       0.56      0.74      0.64        50
           2       0.76      0.65      0.70        34
           3       0.78      0.33      0.47        21
           4       0.68      0.78      0.72        27
           5       0.91      0.80      0.85        25

    accuracy                           0.68       157
   macro avg       0.74      0.66      0.68       157
weighted avg       0.71      0.68      0.68       157



In [ ]:
print(classification_report(train_y, svm_train_pred))

              precision    recall  f1-score   support

           1       0.97      0.87      0.92       158
           2       0.98      0.98      0.98       119
           3       0.98      0.98      0.98        91
           4       0.85      0.97      0.91       127
           5       1.00      1.00      1.00       133

    accuracy                           0.95       628
   macro avg       0.96      0.96      0.96       628
weighted avg       0.96      0.95      0.95       628



In [ ]:
from sklearn import linear_model

In [ ]:
clf_lr=linear_model.LogisticRegression(multi_class='ovr', solver='liblinear')

In [ ]:
clf_lr.fit(train_x_vectors, train_y)

LogisticRegression(multi_class='ovr', solver='liblinear')

In [ ]:
lr_valid_pred=clf_lr.predict(valid_x_vectors)

In [ ]:
lr_train_pred=clf_lr.predict(train_x_vectors)

In [ ]:
lr_valid_pred[0]

3

In [ ]:
print(classification_report(valid_y, lr_valid_pred))

              precision    recall  f1-score   support

           1       0.69      0.58      0.63        50
           2       0.65      0.82      0.73        34
           3       0.86      0.57      0.69        21
           4       0.62      0.78      0.69        27
           5       0.79      0.76      0.78        25

    accuracy                           0.69       157
   macro avg       0.72      0.70      0.70       157
weighted avg       0.71      0.69      0.69       157



In [ ]:
print(classification_report(train_y, lr_train_pred))

              precision    recall  f1-score   support

           1       0.94      0.85      0.89       158
           2       0.94      0.97      0.96       119
           3       0.97      0.93      0.95        91
           4       0.86      0.95      0.91       127
           5       1.00      1.00      1.00       133

    accuracy                           0.94       628
   macro avg       0.94      0.94      0.94       628
weighted avg       0.94      0.94      0.94       628



In [ ]:
test_x_vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
lr_test_pred=clf_lr.predict(test_x_vectors)

In [ ]:
lr_test_pred[0]

3

In [ ]:
test['label']=lr_test_pred

In [ ]:
test.head()

,id,Text,label
0,0,I'm worried about the impact of panic on my re...,3
1,1,Trying to maintain my equilibrium when emotion...,4
2,2,Feeling like I'm in a constant struggle to fin...,1
3,3,I'm quick to dismiss others' opinions as less ...,5
4,4,I hate the person I become when I'm consumed b...,4


In [ ]:
test_without_text=test.drop('Text', axis=1).copy()

In [ ]:
test_without_text.head()

,id,label
0,0,3
1,1,4
2,2,1
3,3,5
4,4,4


In [ ]:
# test_without_text.to_csv('submission.csv', index=False)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
clf_gnb=MultinomialNB().fit(train_x_vectors,train_y)

In [ ]:
gnb_valid_pred=clf_gnb.predict(valid_x_vectors)

In [ ]:
gnb_valid_pred[0]

3

In [ ]:
print(classification_report(valid_y, gnb_valid_pred))

              precision    recall  f1-score   support

           1       0.68      0.50      0.57        50
           2       0.58      0.85      0.69        34
           3       0.90      0.43      0.58        21
           4       0.58      0.81      0.68        27
           5       0.86      0.76      0.81        25

    accuracy                           0.66       157
   macro avg       0.72      0.67      0.67       157
weighted avg       0.70      0.66      0.66       157



In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
clf_knn=KNeighborsClassifier(n_neighbors=8,weights='distance')
clf_knn.fit(train_x_vectors, train_y)

KNeighborsClassifier(n_neighbors=8, weights='distance')

In [ ]:
knn_valid_pred=clf_knn.predict(valid_x_vectors)

In [ ]:
knn_valid_pred[0]

1

In [ ]:
print(classification_report(valid_y, knn_valid_pred))

              precision    recall  f1-score   support

           1       0.47      0.78      0.59        50
           2       0.75      0.35      0.48        34
           3       0.67      0.38      0.48        21
           4       0.68      0.70      0.69        27
           5       0.94      0.68      0.79        25

    accuracy                           0.61       157
   macro avg       0.70      0.58      0.61       157
weighted avg       0.67      0.61      0.60       157

